<div>
<img src="figures/svtLogo.png"/>
</div>  

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Lab 8 - Elimination of variables, Penalty and SQP methods</h2></center>

$$\newcommand{\bx}{\mathbf{x}}$$
The following problem is given:
\begin{align*}
  \min_{\bx \in \mathbb{R}^2} \;\; & f(\bx)  \\
   \text{s.t.} \;\;& x_1+x_2=8,
\end{align*}

where $f(\bx) = - (x_1^2+x_2^2+4x_1x_2)$.

<u>Task 1</u>: Find the minimum of the function using variable elimination.
Check the second-order sufficient conditions for the unconstrained one-variable problem.

<details>
We solve for $x_1$ (we could also solve for $x_{2}$). This implies

\begin{equation} \label{resolve}
	x_1 = 8-x_2.
\end{equation}
    
The objetcive function can now be written as $f(x_1,x_2) = -(8-x_2)^2 -x_2 ^2 - 4 (8-x_2) x_2 = 2 \,x_2^2 - 16 x_2 -64$. 
<br>    
We define the function $S(x_2) = x_2^2 -8 x_2$. Constant terms (the number $64$, or the multiplication factor of $2$) are redundant in the objective function. Hence, we can formulate the unconstrained optimization problem as 
    
\begin{equation} \label{varEli}
		\min_{x_2} x_2 ^2 -8\, x_2 \,.
\end{equation}
   
In order to get the stationary point, we differentiate $S(x_2)$ with respect to $x_2$ and set it to zero. This results in $x_2 ^* = 4$. From the equality constraint, we get $x_1 ^* = 4$.
<br>         
The Hessian of the one-variable problem is $S_{x_2x_2} \equiv 2$, i.e., the Hessian is positive definite and the second-order sufficient conditions are satisfied.
    
</details>

Another possibility is to use the following penalty function:
\begin{align*}
  	Q(\bx;\mu)=f(\bx)+\frac{1}{2\mu} (x_1+x_2-8)^2\,,
\end{align*}
with $\mu>0$ being a penalty parameter.
     
<br>
<u>Task 2</u>: Write down the first-order necessary condition of optimality for minimizing $Q$.

<details>

We differentiate $Q$ with
respect to $x_1$ and $x_2$.
<br>
<br>
$$\begin{aligned}
\frac{\partial Q}{\partial x_1} (x_1,x_2) = -2 x_1 - 4 x_2 + \frac{1}{\mu} (x_1 + x_2 -8) \\
\frac{\partial Q}{\partial x_2} (x_1,x_2) = -2 x_2 - 4 x_1 + \frac{1}{\mu} (x_1 + x_2 -8)
\end{aligned}$$
<br>
Now, we have to set the partial derivatives to zero for the first-order necessary condition of optimality:
<br>
<br>
$$\left(
  \begin{array}{cc}
    \frac{1}{\mu} - 2 & \frac{1}{\mu} - 4 \\
    \frac{1}{\mu} - 4 & \frac{1}{\mu} - 2 \\
  \end{array}
\right)
\left(
  \begin{array}{c}
    x_1 \\
    x_2 \\
  \end{array}
\right)
=
\left(
  \begin{array}{c}
    \frac{8}{\mu} \\
    \frac{8}{\mu} \\
  \end{array}
\right)$$

</details>

<u>Task 3</u>: What happens if $\mu \downarrow 0$?  Complete the implementation of the quadratic penalty method below:

Also, report the eigenvalues and the condition number of the Hessian for each $\mu$.

In [3]:
import numpy as np
import scipy.optimize as sp

# to calculate the gradient and Hessian of the objective function
from autograd import grad
from autograd import hessian

# to solve additionally using SLSQP solver, later on
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint
from math import inf

### Objective, constraint, quadratic penalty function, gradient and hessian

In [4]:
def objective(X):
    x1, x2 = X[0], X[1]
    f = -(x1 ** 2 + x2 ** 2 + 4 * x1 * x2)
    return f

In [5]:
def constraint(X):
    x1, x2 = X[0], X[1]
    c = x1 + x2 -8
    return c

In [6]:
def penaltyFunction(X, mu):
    x1, x2 = X[0], X[1]
    f = -(x1 ** 2 + x2 ** 2 + 4 * x1 * x2) + 1 / (2 * mu) * (x1 + x2 -8) ** 2
    return f

In [7]:
def gradient_function(x, mu): 
    return [el.item() for el in grad(penaltyFunction, 0)(x, mu)]

In [8]:
def hessian_function(x, mu): 
    return hessian(penaltyFunction, 0)(x, mu)

### Quadratic penalty method

In [9]:
def qpm(x0, mu): 
    
    # get eigenvalues of the Hessian
    w, v = np.linalg.eig(hessian_function(x0, mu))
    
    # get condition number of the Hessian
    n = np.linalg.cond(hessian_function(x0, mu))
    
    # unconstrained optimization using BFGS method
    res = sp.minimize(penaltyFunction, x0, args=(mu), method='BFGS', jac=gradient_function)
    
    return w, n, res.x

In [10]:
mu = 1
x0 = np.array([1.,1.])

# acceptable constraint violation at optimum
vol = 1e-15
constraint_violation = True

it = 0

print ("{:<10} {:<10} {:<20} {:^20} {:^30}".format('iter','mu','minimum','condition nr.', 'constraint violation'))
while constraint_violation:
    it=it+1
    
    w, n, xmin = qpm(x0,mu)
    print ("{:<10d} {:<10.3e} [{:^8.4f}, {:^8.4f}] {:<4} {:<20.2e} {:^20.3e}".format(it,mu,xmin[0],xmin[1],' ',n,constraint(xmin)))
    
    if constraint(xmin)<=vol:
        constraint_violation = False 
    
    # update for next iteration
    mu = mu/2
    x0 = xmin

iter       mu         minimum                 condition nr.          constraint violation     
1          1.000e+00  [732.3181, 732.3181]      2.00e+00                  1.457e+03      
2          5.000e-01  [1463.6362, 1463.6362]      1.00e+00                  2.919e+03      
3          2.500e-01  [16.0000 , 16.0000 ]      1.00e+00                  2.400e+01      
4          1.250e-01  [ 6.4000 ,  6.4000 ]      5.00e+00                  4.800e+00      
5          6.250e-02  [ 4.9231 ,  4.9231 ]      1.30e+01                  1.846e+00      
6          3.125e-02  [ 4.4138 ,  4.4138 ]      2.90e+01                  8.276e-01      
7          1.562e-02  [ 4.1967 ,  4.1967 ]      6.10e+01                  3.934e-01      
8          7.812e-03  [ 4.0960 ,  4.0960 ]      1.25e+02                  1.920e-01      
9          3.906e-03  [ 4.0474 ,  4.0474 ]      2.53e+02                  9.486e-02      
10         1.953e-03  [ 4.0236 ,  4.0236 ]      5.09e+02                  4.715e-02      
11 

### SLSQP method (scipy)

We will solve the problem using scipy's SLSQP solver (written by Dieter Kraft, DLR Oberpfaffenhofen)

In [11]:
x0 = np.array([1.,1.])

bounds = Bounds([-inf,-inf], [inf,inf])

# The constraint is actually linear, so you can try a different approach.
# See SLSQP documentation for more details on how to set up linear constraints.
nonlinear_constraints = NonlinearConstraint(constraint, 0, 0)

res = sp.minimize(objective, x0, method='SLSQP',
               constraints=[nonlinear_constraints], bounds=bounds)

print("minimum = {}".format(res.x))
print("constraint violation = {}".format(constraint(res.x)))

minimum = [4. 4.]
constraint violation = -1.7763568394002505e-15
